In [ ]:
%matplotlib inline
from arc_maps import get_data
from smif.data_layer import Results
import seaborn as sbn
import matplotlib.pyplot as plt
import numpy as np
results = Results({'interface': 'local_csv', 'dir': '../'})

sbn.set_context('talk', font_scale=0.8, rc={"lines.linewidth": 2.5})
sbn.set_palette('Paired')

In [ ]:
results.available_results('arc_ets__baseline')

In [ ]:
results.list_sector_models('arc_ets__baseline')

In [ ]:
elec_outputs = [x for x in results.list_outputs('energy_demand_constrained') if 'electricity' in x.split("_")]

model_run_names = ['arc_ets__new-cities', 'arc_ets__expansion']
model_names = ['energy_demand_constrained']
timesteps=[2015, 2030, 2050]

df = results.read_results(
        model_run_names, model_names, elec_outputs, timesteps).drop(
    columns=['decision'])

In [ ]:
total_elec_demand = df.set_index(['model_run', 'timestep', 'lad_uk_2016', 'hourly']).sum(axis=1).reset_index()

In [ ]:
national_elec_demand = total_elec_demand.groupby(
    by=['model_run', 'timestep', 'hourly']).sum().rename(columns={0:'elec_demand'})

In [ ]:
data = national_elec_demand.reset_index().melt(id_vars=['model_run', 'timestep', 'hourly'])
data['model_run'] = data['model_run'].str.replace('arc_ets__', '')
filtered = data[data['hourly'].isin(range(0,168))]
fig = sbn.relplot(data=filtered, x='hourly', col='timestep', y='value', row='model_run', kind='line')
fig.savefig('ed_electricity.png', transparent=True)

# Arc LADs: Peak demand

In [ ]:
def highlight(data, id_col='geo_label'):
    
    def group_hlads(index):
        if index in {'Bedford', 'Cambridge', 'Milton Keynes', 'Oxford', 
                     'Aylesbury Vale', 'Central Bedfordshire', 'Huntingdonshire', 'South Cambridgeshire'}:
            return index
        else:
            return 'Other'
    
    data['summary_lads'] = data[id_col].apply(group_hlads) 
    
    return data

In [ ]:
filepath = '../credential.ini'
arc_lads = get_data.get_lads(filepath)

In [ ]:
df['model_run'] = df['model_run'].str.replace('arc_ets__', '')
elec_sum = df.set_index(['model_run', 'timestep', 'lad_uk_2016', 'hourly']).sum(axis=1).reset_index()
elec_in_arc = arc_lads.join(elec_sum.set_index('lad_uk_2016')).reset_index().drop(columns=['geometry','gid'])
elec_in_arc = highlight(elec_in_arc)[['summary_lads','model_run','timestep', 'hourly', 0]]
peak = elec_in_arc.groupby(['summary_lads','model_run','timestep']).max().rename(
    columns={0: 'peak_demand'}).drop(columns='hourly')

In [ ]:
ax = sbn.relplot(data=peak.reset_index(), x='timestep', y='peak_demand', 
                 col='model_run', hue='summary_lads')
ax.set_axis_labels('year', 'peak demand (GW)')
ax.savefig('ed_peak_demand.png', transparent=True)

# Arc LADs Total Demand

In [ ]:
elec_in_arc = arc_lads.join(elec_sum.set_index('lad_uk_2016')).reset_index().drop(columns=['geometry','gid'])
elec_in_arc = elec_in_arc.rename(columns={0: 'peak_demand'})
total_elec_dem = elec_in_arc.groupby(by=['model_run', 'timestep', 'geo_label']).sum().drop(columns='hourly')

In [ ]:
total_elec_dem = highlight(total_elec_dem.reset_index())
ax = sbn.relplot(data=total_elec_dem.reset_index(), x='timestep', y='peak_demand', 
                 col='model_run', hue='summary_lads', kind='line')
ax.set_axis_labels('year', 'total elec demand (GWh)')
ax.savefig('ed_total_demand.png', transparent=True)

In [ ]:
highlight(total_elec_dem.reset_index())